In [1]:
import pandas as pd

In [2]:
# Reading the files
data = pd.read_csv("train.csv")
testdata = pd.read_csv("test.csv")
# Separating the passengerid for later purposes
testids = testdata["PassengerId"]

def cleaning(datas):
    #Getting rid of irrelevant data
    datas = datas.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)
    #Do a for in loop that fills the blank spaces with median of column
    columns = ["SibSp", "Parch", "Fare", "Age"]
    for column in columns:
        datas[column].fillna(datas[column].median(), inplace=True)
    #Filling the empty spaces with U, U = Unknown
    datas.Embarked.fillna("U", inplace=True)
    return datas
#Perform our function to our data sets
data = cleaning(data)
testdata = cleaning(testdata)

In [3]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
# Choosing the columns
columns = ["Sex", "Embarked"]
# Doing a for in loop that takes our columns and tranforms the data to ones and zeros
# It will help processing
for column in columns:
    data[column] = le.fit_transform(data[column])
    testdata[column] = le.transform(testdata[column])
    print(le.classes_)
    
data.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# Delete the survived column
survived = data["Survived"]
withoutsurvived = data.drop("Survived", axis=1)
#Train the data
withoutsurvived_train, withoutsurvived_val, survived_train, survived_val = train_test_split(withoutsurvived, survived, test_size=0.2, random_state=42)

In [6]:
# Creating a classifier with maximum iterations
classifier = LogisticRegression(random_state=0, max_iter=1000).fit(withoutsurvived_train, survived_train)

In [7]:
# Prediction of the data
preds = classifier.predict(withoutsurvived_val)
from sklearn.metrics import accuracy_score
# Check the accuracy score
accuracy_score(survived_val, preds)

0.8100558659217877

In [8]:
# Creating the predictions from test data for submission
submitpreds = classifier.predict(testdata)

In [9]:
#Creating a dataframe of our collected data
dataframe = pd.DataFrame({"PassengerId":testids.values,
                       "Survived": submitpreds,
                        })

In [10]:
# Creating the file from our dataframe
dataframe.to_csv("answer.csv", index=False)